# Generating an LLM Agent Routine Using Meta-Prompting

This notebook utilizes **meta-prompting techniques** to generate a structured **system prompt**, providing the LLM Agent with a clear **operational blueprint**. This ensures the agent functions as an **autonomous, policy-compliant assistant**, capable of making informed decisions within its designated domain.

## Key Inputs

1. **Policy Document (Operational Manual)** – A structured set of operational procedures, rules, and best practices followed by human operators. This defines the expected standards for decision-making and compliance.
2. **Available Tools List** – A catalog of external tools the LLM Agent can utilize to interact with various systems, including APIs, databases, and automation services. Each tool has specific capabilities, inputs, and constraints.

## Objectives

By synthesizing these inputs, we will construct a **system prompt** that defines a structured **routine** for the LLM Agent to follow. This routine will:  

- **Ensure compliance with operational policies**, maintaining consistency and adherence to best practices.  
- **Optimize tool usage**, leveraging available functionalities efficiently while respecting their constraints.  
- **Implement a structured reasoning framework**, enabling the agent to process information, make decisions, and execute tasks in a logical and context-aware manner.  

<div align="center">
  <img src="images/generate_ai.png" alt="Meta-prompting Generate" width="800">
</div>

## Getting Started

Prerequisites:

- Python 3.11 or 3.12 installed on your system.
- LLM Agentic Tool Mesh library installed. If not, install it using: `pip install 'llmesh[all]'`
- Install kernel for Jupiter notebook `pip install ipykernel`
- Create `.env` file inside `notebooks/data` folder with the value of `PYTHONPATH` and `OPENAI_API_KEY` environment variables

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file located in the Data folder
dotenv_path = os.path.join('data', '.env')
load_dotenv(dotenv_path)

# Read environment variables
python_path = os.getenv('PYTHONPATH')
if not python_path:
    raise ValueError("Project path is not set.")
llm_api_key = os.getenv('OPENAI_API_KEY')
if not llm_api_key:
    raise ValueError("API key is not set in environment variables.")

# Change working directory
os.chdir(python_path)

## Loading the Policy Document

The Policy Document (Operational Manual) is read from a Markdown file and serves as the foundation for defining the LLM Agent’s operational guidelines.  

In [2]:
from notebooks.utils import read_policy
from IPython.display import Markdown

policy = read_policy()
display(Markdown(policy))

# **Private 5G Network Management Policy**

**Purpose**: This document serves as a detailed guide for internal network administrators and support teams to manage a private 5G network, namely HPE 5G Core. The focus is on ensuring efficient operations, maintaining security, optimizing performance, and handling user requests effectively.

**Note**: Always follow best practices in cybersecurity, compliance, and performance optimization while managing the private 5G network.

## **Table of Contents**

1. General Guidelines for Network Management
2. Access Control and User Management
3. Network Configuration and Optimization
4. FAQs for Common Scenarios

### **1. General Guidelines for Network Management**

* **Monitor Network Performance**: Continuously track key performance indicators (KPIs) such as latency, bandwidth usage, and signal strength within the HPE 5G Core.
* **Monitor AMF-NGAP Status**: Retrieve the current status of the N2 interface between the gNB and the AMF within the HPE 5G Core.
* **Retrieve Logs from Network Functions**: Obtain logs for a specific network function within the HPE 5G Core.
* **Read Network Function Profile**: Access the profile of a specific network function within the HPE 5G Core.

### **2. Access Control and User Management**

#### **2.1 Device Management**

* **Approved Device List**: List of authorized devices/users/UEs, defined by the 'imsi-', permitted to access the HPE 5G Core.
* **Device Information**: Details of authorized device/user/UE within the HPE 5G Core. If a user/device/UE doesn't start with "imsi-", use the request details to refer to the approved device list and find the corresponding entry.
* **Approved Data Profiles**: List of authorized data profiles, defined by the uuid, within the HPE 5G Core.
* **Data Profile Information**: Details of a specific data profile within the HPE 5G Core. If the specified data profile does not exist, use the available information in the request to locate it in the approved data profiles list.
* **Retrieve UE Location Information**: Obtain the location of a target UE within the HPE 5G Core.

### **3. Network Configuration and Optimization**

* **Assign New User Data Profile**: Allocate a dedicated network data profile for different use cases. The profile must be from the authorized list, and the user must be on the approved device/user/UE list. If the user/device/UE doesn’t start with "imsi-", use the request details to find it in the approved device list. If the data profile doesn’t exist, use the request details to find it in the approved data profiles list.

### **4. FAQs for Common Scenarios**

**Q1: How do I retrieve the list of authorized users?**

* **Policy**: Submit a device list request to retrieve the authorized list of devices/users/UEs within the HPE 5G Core, which relies on UDM.

**Q2: How do I retrieve the list of data profiles?**

* **Policy**: Submit a device list request to retrieve the authorized list of data profiles within the HPE 5G Core, which relies on UDM.

**Q3: How do I assign a new data profile to a user?**

* **Policy**: Submit a data profile assignment request. The profile must belong to the authorized list of data profiles as well as the user must be in the authorized list of devices/users/UEs.

**Q4: How do I retrieve information for a specific user?**

* **Policy**: Submit a device information request. The user must be in the authorized list of devices/users/UEs.

**Q5: How do I retrieve information about the NGAP status?**

* **Policy**: Submit a Monitor AMF-NGAP Status request to obtain the current status of the N2 interface between the gNB and the AMF.

**Q6: How do I monitor network performance?**

* **Policy**: Submit a Monitor Network Performance request to track key performance indicators (KPIs).

**Q7: How do I obtain logs for a specific network function?**

* **Policy**: Submit a Retrieve Logs from NF request, specifying the network function for which logs are required.

**Q8: How do I retrieve a user's location information?**

* **Policy**: Submit a Retrieve UE Location Information request, specifying the target user.

**Q9: How do I retrieve the profile of a specific network function?**

* **Policy**: Submit a Read Network Function Profile request, specifying the network function instance to retrieve its profile.

**Q10: How do I retrieve information for a specific data profile?**

* **Policy**: Submit a Data Profile Information request, specifying the data profile instance to retrieve its profile.


## Loading the System Prompt for Meta-Prompting

The system prompt is loaded from a predefined prompt file and enriched by appending the tool descriptions to ensure seamless integration with the LLM Agent's capabilities.  

In [3]:
import json
from notebooks.utils import load_prompt, TOOLS, TOOL_CLOSE

tools = f"\n\n```json\n{json.dumps(TOOLS, indent=4)}\n```"
system_prompt = load_prompt('routine_generation', {'tools':tools, 'tool_close': TOOL_CLOSE})
display(Markdown(system_prompt))

2025-02-18 16:20:13,326 - ATHON - DEBUG - Prompt generated from notebooks/prompts//routine_generation.txt with params {'tools': '\n\n```json\n[\n    {\n        "name": "get_amf_ngap_status",\n        "description": "Retrieve AMF-NGAP status from N2 interface.",\n        "parameters": {\n            "nf": {\n                "description": "Network function name where take the AMF-NGAP status from N2 interface.",\n                "type": "string",\n                "required": true\n            }\n        }\n    },\n    {\n        "name": "patch_provisioned_data_profile",\n        "description": "Allocate a dedicated network data profile for different use cases(e.g., IoT, enterprise applications, emergency services). The assigned data profile must belong to the authorized list of available profiles,the user must be on the authorized list of devices/users.",\n        "parameters": {\n            "nf": {\n                "description": "The name of network function where take the profiles."

You are a helpful assistant tasked with taking an external facing operational guide and converting it into a internal-facing programmatically executable routine optimized for an LLM. 
The LLM using this routine will be tasked with reading the policy, answering incoming questions from customers, and helping drive the case toward resolution.

Please follow these instructions:
1. **Review the operational guide policy carefully** to ensure every step is accounted for. It is crucial not to skip any steps or policies.
2. **Organize the instructions into a logical, step-by-step order**, using the specified format. 
3. **Use the following format**:
   - **Main actions are numbered** (e.g., 1, 2, 3).
   - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
      **Sub-actions should start on new lines**
   - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was purchased within 30 days, then...').
   - **For instructions that require more information from the customer**, provide polite and professional prompts to ask for additional information.
   - **For actions that require data from external systems**, write a step to call a function using backticks for the function name (e.g., call the `check_delivery_date` function).
      - **If a step requires the LLM agent to take an action** (e.g., check a metric), generate a function call for this action (e.g., call the `get_metric` function).
      - **Only use the available set of functions that are defined below.
   - **If there is an action an assistant can perform on behalf of the user**, include a function call for this action (e.g., call the `change_email_address` function), and ensure the function is defined with its purpose and required parameters.
      - **Only use the available set of functions that are defined below.
   - **The step prior to case resolution should always be to ask if there is anything more you can assist with**.
   - **End with a final action for case resolution**: calling the `close_case` function should always be the final step.
4. **Ensure compliance** by making sure all steps adhere to company policies, privacy regulations, and legal requirements.
5. **Handle exceptions or escalations** by specifying steps for scenarios that fall outside the standard policy.
6. **Ensure coverage** by checking that all of the conditions covered in the policy are also covered in the routines

**Important**: Always wrap the functions you return in backticks i.e. `check_ticket_type`. Do not include the arguments to the functions.

Here are the currently available set of functions in JSON format: 
TOOLS: 

```json
[
    {
        "name": "get_amf_ngap_status",
        "description": "Retrieve AMF-NGAP status from N2 interface.",
        "parameters": {
            "nf": {
                "description": "Network function name where take the AMF-NGAP status from N2 interface.",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "patch_provisioned_data_profile",
        "description": "Allocate a dedicated network data profile for different use cases(e.g., IoT, enterprise applications, emergency services). The assigned data profile must belong to the authorized list of available profiles,the user must be on the authorized list of devices/users.",
        "parameters": {
            "nf": {
                "description": "The name of network function where take the profiles.",
                "type": "string",
                "required": true
            },
            "supi": {
                "description": "The id which specify the supi starting with 'imsi-'.",
                "type": "string",
                "required": true
            },
            "uuid": {
                "description": "The id which specify the data profile.",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "get_dataprofiles",
        "description": "Get List of authorized data profiles within the HPE 5G Core.",
        "parameters": {
            "nf": {
                "description": "The name of network function where take list of authorized data profiles.",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "get_logs",
        "description": "Retrieve network function logs.",
        "parameters": {
            "nf": {
                "description": "The name of network function where take the logs.",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "get_supi_location",
        "description": "Obtain the location of a target UE within the HPE 5G Core.",
        "parameters": {
            "nf": {
                "description": "The name of network function where take the supi location.",
                "type": "string",
                "required": true
            },
            "supi": {
                "description": "The id which specify the supi starting with 'imsi-'.",
                "type": "string",
                "required": true
            },
            "pdu_session_id": {
                "description": "The id which specify the PDU Session ID.",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "get_dataprofile",
        "description": "Get a specific and authorized data profiles within the HPE 5G Core.",
        "parameters": {
            "nf": {
                "description": "The name of network function where take the specific data profile.",
                "type": "string",
                "required": true
            },
            "uuid": {
                "description": "The id which specify the data profile.",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "get_supi",
        "description": "Get specific user/device permitted to access the HPE 5G Core.",
        "parameters": {
            "nf": {
                "description": "The name of network function where take the specific supi.",
                "type": "string",
                "required": true
            },
            "supi": {
                "description": "The id which specify the supi starting with 'imsi-'.",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "get_supis",
        "description": "Get List of authorized devices/users permitted to access the HPE 5G Core.",
        "parameters": {
            "nf": {
                "description": "The name of network function where take list of supis (authorized devices).",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "get_network_performance",
        "description": "Retrieve a specific network performance metric.",
        "parameters": {
            "nf": {
                "description": "The name of network function to be measured",
                "type": "string",
                "required": true
            },
            "metric": {
                "description": "The metric to be measured",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "get_nf_profile",
        "description": "Access the profile status of a specific network function within the HPE 5G Core.",
        "parameters": {
            "nf": {
                "description": "The name of network function to be used to retrieve the profile",
                "type": "string",
                "required": true
            },
            "target_nf": {
                "description": "The name of network function of the profile",
                "type": "string",
                "required": true
            }
        }
    },
    {
        "name": "close_case",
        "description": "Tool to close a case.",
        "parameters": {
            "nf": {
                "description": "The name of network function to be used to retrieve the profile",
                "type": "string",
                "required": true
            }
        }
    }
]
```

Please convert the following operational guide policy into the formatted routine, ensuring it is easy to follow and execute programmatically. Ensure that you **only** use the functions provided and do **not** create net new functions.

## Generating the Routine

The routine is generated by applying meta-prompting techniques to structure the LLM Agent’s operational flow, ensuring alignment with policy guidelines and effective tool utilization.  

In [4]:
from notebooks.utils import calculate_routine

routine = calculate_routine(system_prompt, policy)
display(Markdown(routine))

2025-02-18 16:20:13,332 - ATHON - INFO - Data loaded from notebooks/files/p5g_routine_0_cached.pkl.


1. Identify the type of customer request by parsing the input for keywords corresponding to the following categories:
   a. If the request mentions monitoring network performance (KPIs such as latency, bandwidth, signal strength), then proceed to step 2.
   b. If the request is about retrieving the AMF-NGAP status, then proceed to step 3.
   c. If the request asks for network function logs, then proceed to step 4.
   d. If the request is to read a network function profile, then proceed to step 5.
   e. If the request is about obtaining the list of authorized users/devices/UEs, then proceed to step 6.
   f. If the request is about retrieving information for a specific user/device/UE, then proceed to step 7.
   g. If the request is to retrieve the list of approved data profiles, then proceed to step 8.
   h. If the request is for details of a specific data profile, then proceed to step 9.
   i. If the request is to assign a new data profile to a user, then proceed to step 10.
   j. If the request is to obtain UE location information, then proceed to step 11.
   k. Else, if the request does not match any of the above, politely prompt the customer for clarification and escalate if necessary.

2. Monitor Network Performance:
   a. Prompt the customer: “Please provide the network function name and the specific KPI metric you want to track (e.g., latency, bandwidth, signal strength).”
   b. Then call the `get_network_performance` function.

3. Retrieve AMF-NGAP Status:
   a. Prompt the customer: “Please provide the network function name where the AMF-NGAP status is required.”
   b. Then call the `get_amf_ngap_status` function.

4. Retrieve Logs from a Network Function:
   a. Prompt the customer: “Please specify the network function name from which you require the logs.”
   b. Then call the `get_logs` function.

5. Read Network Function Profile:
   a. Prompt the customer: “Please provide the network function name for which you need the profile.”
   b. Then call the `get_nf_profile` function.

6. Retrieve the List of Authorized Users/Devices/UEs:
   a. Prompt the customer: “Please confirm if you want the list of all authorized devices/users (those with IDs starting with ‘imsi-’) accessed in the HPE 5G Core.”
   b. Then call the `get_supis` function.

7. Retrieve Information for a Specific User/Device/UE:
   a. Prompt the customer: “Please provide the identifier for the specific user/device/UE. Ensure that the ID starts with ‘imsi-’. If not, please re-check the approved device list.”
   b. Then call the `get_supi` function.

8. Retrieve the List of Authorized Data Profiles:
   a. Prompt the customer: “Please provide the network function name to retrieve the list of authorized data profiles.”
   b. Then call the `get_dataprofiles` function.

9. Retrieve Specific Data Profile Information:
   a. Prompt the customer: “Please provide the UUID of the data profile you want to retrieve.”
   b. Then call the `get_dataprofile` function.

10. Assign a New User Data Profile:
   a. Prompt the customer: “Please provide the network function name, the user’s ID (must start with ‘imsi-’ or match an entry from the approved device list), and the data profile’s UUID (from the approved list).”
   b. If the user’s identifier does not start with “imsi-”, then ask: “Can you confirm the identifier or provide additional details to locate the device in the approved list?” 
   c. If the specified data profile does not exist in the approved data profiles, then ask: “The provided data profile UUID is not recognized. Can you please verify or provide additional details?”
   d. Once all required information is verified, then call the `patch_provisioned_data_profile` function.

11. Retrieve UE Location Information:
   a. Prompt the customer: “Please provide the network function name, the user/device/UE ID (must start with ‘imsi-’), and the PDU Session ID for which you require the location information.”
   b. Then call the `get_supi_location` function.

12. After completing the requested action(s), always ask the customer:
   a. “Is there anything more I can assist you with regarding managing the HPE 5G Core?”

13. Final Case Resolution:
   a. Once the customer confirms that no further assistance is needed, call the `close_case` function.